https://www.analyticsvidhya.com/blog/2020/11/words-that-matter-a-simple-guide-to-keyword-extraction-in-python/?

Term Frequency – How frequently a term occurs in a text. It is measured as the number of times a term t appears in the text / Total number of words in the document

Inverse Document Frequency – How important a word is in a document. It is measured as log(total number of sentences / Number of sentences with term t)

TF-IDF – Words’ importance is measure by this score. It is measured as TF * IDF

In [ ]:
pip install nltk

1. Import packages. Need to install all nltk packages for everything to work.

In [ ]:
import nltk
import math
import bs4
import requests
from nltk import tokenize
from operator import itemgetter
from bs4 import BeautifulSoup
from pathlib import Path
nltk.download("all")

Load text from a website

In [ ]:
link = 'https://playbooks.idmanagement.gov/arch/services/'
html_content = requests.get(link).text
soup = BeautifulSoup(html_content, "html.parser")
doc = soup.find('main')
print(doc)

2. Created a text file of loaded words from website. Declare variables. Find total words in the document.

In [ ]:
# doc = Path('ficamarchkeywords.txt').read_text()
doc = Path('jobs-20220323.yml').read_text()
doc = doc.replace('\n','')
print(doc)
total_words = doc.split()
total_word_length = len(total_words)
print(total_word_length)

3. Remove stop words.

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
stop_words = set(stopwords.words('english'))
word_tokenize(doc)

find total number of sentences

In [ ]:
total_sentences = tokenize.sent_tokenize(doc)
total_sent_len = len(total_sentences)
print(total_sent_len)

Calculate term frequency for each word

In [ ]:
tf_score = {}
for each_word in total_words:
    each_word = each_word.replace('.','')
    if each_word not in stop_words:
        if each_word in tf_score:
            tf_score[each_word] += 1
        else:
            tf_score[each_word] = 1

# Dividing by total_word_length for each dictionary element
tf_score.update((x, y/int(total_word_length)) for x, y in tf_score.items())
print(tf_score)

Function to check if word is present in a sentence list

In [ ]:
def check_sent(word, sentences): 
    final = [all([w in x for w in word]) for x in sentences] 
    sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
    return int(len(sent_len))

Calculate inverse document frequency for each word

In [ ]:
idf_score = {}
for each_word in total_words:
    each_word = each_word.replace('.','')
    if each_word not in stop_words:
        if each_word in idf_score:
            idf_score[each_word] = check_sent(each_word, total_sentences)
        else:
            idf_score[each_word] = 1

# Performing a log and divide
idf_score.update((x, math.log(int(total_sent_len)/y)) for x, y in idf_score.items())

print(idf_score)

Calculate tf * idf

In [ ]:
tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}
print(tf_idf_score)

Create a function to get N important words in the document.

In [ ]:
def get_top_n(dict_elem, n):
    result = dict(sorted(dict_elem.items(), key = itemgetter(1), reverse = True)[:n]) 
    return result

Get the top 10 words of significance

In [ ]:
print(get_top_n(tf_idf_score, 10))